In [ ]:
# nullpeptide_extraction/extract_peptides.py

import re,os,sys,glob
import mycustom2
from Bio.Seq import Seq 
from collections import Counter


def reader(path):
        datafile=open(path,"r")
        data=datafile.readlines()
        datafile.close()
        Data=[]
        for i in data:
                Data+=[i.strip().split('\t')]
        return Data

def fast_counter(SequencesL, kmer_lengthL): 
    Counts = Counter()
    for kmer_length in kmer_lengthL:
    	for seq in SequencesL:
        	for i in range(0, len(seq)+1-kmer_length):
        	    motif = seq[i:i+kmer_length]
        	    Counts[motif] += 1
    return Counts.most_common()


filed = "UP000005640_9606.fasta"
sequenceO = mycustom2.FastaFile(filed)
sequencesL = [ i.sequence.upper() for i in sequenceO ]

# Peptide lengths to scan
kmer_lengthL=range(1,8)
Counts=fast_counter(sequencesL,kmer_lengthL)

datafile=open("one_to_seven_amino_acids_UP000005640_9606","w")
for i in Counts:
	datafile.write(str(i[0])+'\t'+str(i[1])+'\n')
datafile.close()	

In [ ]:
# nullpeptide_extraction/generate_all_counts.py

import re,os,sys,glob
import mycustom 
from Bio.Seq import Seq 
from collections import Counter
import pickle
import io


def reader(path):
        datafile=open(path,"r")
        data=datafile.readlines()
        datafile.close()
        Data=[]
        for i in data:
                Data+=[i.strip().split('\t')]
        return Data

DataL=reader('one_to_seven_amino_acids_UP000005640_9606')
DataD = {}
for i in DataL:
        if "N" not in i[0]:
                DataD[i[0]]=int(i[1])


DataL=reader("peptide_kmers_1_7nt")
Data2D={}
for k in DataL:
        Data2D[k[0]]=0

DataTD= { k: DataD.get(k, 0) + Data2D.get(k, 0) for k in set(DataD) | set(Data2D) }

datafile=open("one_to_seven_amino_acids_UP000005640_9606_total","w")
for i in DataTD.items():
        datafile.write(i[0]+'\t'+str(i[1])+'\n')
datafile.close()

In [ ]:
# nullpeptide_extraction/mycustom2.py

class GenericFile(object):
    def __init__(self, fileNameS):
            try:
                fileF = open(fileNameS)
                self.fileS = fileF.read()
                fileF.close()
            except:
                self.fileS = fileNameS.read()

import pdb

class FastaFile(GenericFile):
    """
    This class represents a FastaFile.
    """

    def __init__(self, fasta, fileName=True):
        """

        :rtype : object
        :param fasta: This can be either a file name (string) or a HTML file upload element.
        :param fileName: This is a boolean value. If False, then it is the fasta string
        :return: does not return anything.
        """
        if fileName == True:
            super(FastaFile, self).__init__(fasta)
        elif fileName == False:
            if ">" not in fasta:
                raise ValueError("It is not in Fasta Format")
            self.fileS = fasta
        else:
            raise Exception

        self.sequences = None
        self.sequences = self.getSequences()


    def __getitem__(self, item):
        if self.sequences is None:
            self.getSequences()
        return self.sequences[item]

    def __len__(self):
        return len(self.getSequences())

    def __repr__(self):
        return "< Fasta file: %s sequences >" % len(self)

    def __str__(self):
        return "< Fasta file: %s sequences >" % len(self)

    def getSequences(self):
        if self.sequences is None:
            firstSplitL = self.fileS.strip().split(">")[1:]
            eachByLineL = [i.splitlines() for i in firstSplitL]

            sequences = []
            for i in eachByLineL:
                header = i[0]
                sequence = "".join(i[1:]).upper()
                sequence = sequence.replace(" ", "")
                sequence = sequence.replace("\t", "")
                if sequence == "":
                    #raise ValueError("There is no sequence !")
		    pass
                sequences.append(Sequence(name=header, sequence=sequence))

            self.sequences = sequences

        return self.sequences

    def getMaxLength(self):
        """
        This returns the max length of a sequence in this object
        """
        sequences = self.getSequences()
        maxLength = 0
        for sequence in sequences:
            seqLength = len(sequence)
            if seqLength > maxLength:
                maxLength = seqLength
        return maxLength

    def areDuplicatesPresent(self):
        sequences = self.getSequences()
        sequenceL = [i.sequence for i in sequences]
        if len(sequenceL) > len(set(sequenceL)):
            return True
        return False

    def getNames(self):

        names = [sequence.name for sequence in self.sequences]
        return names


class Sequence(object):
    """
    This class represents a DNA/RNA sequence.
    Each object needs to have a name.
    """
    def __init__(self, name, sequence):
        self.name = name
        self.sequence = sequence

    def __repr__(self):
        return "%s : %s" %(self.name, self.sequence)

    def __getitem__(self, item):
        return self.sequence[item]

    def __len__(self):
        return len(self.sequence)

    def __str__(self):
        return "%s : %s " % (self.name, self.sequence)

In [ ]:
# nullomer_extraction scripts

In [1]:
# nullomer_extraction/extract_kmers.py

import re,os,sys,glob
import mycustom 
from Bio.Seq import Seq 
from collections import Counter
import pickle


def reader(path):
        datafile=open(path,"r")
        data=datafile.readlines()
        datafile.close()
        Data=[]
        for i in data:
                Data+=[i.strip().split('\t')]
        return Data

def fast_counter(SequencesL,SequencesL_rev_compl, kmer_lengthL): 
    Counts = Counter()
    for kmer_length in kmer_lengthL:
    	for seq in SequencesL:
        	for i in range(0, len(seq)+1-kmer_length):
        	    motif = seq[i:i+kmer_length]
        	    Counts[motif] += 1
        for seq2 in SequencesL_rev_compl:
                for im in range(0, len(seq2)+1-kmer_length):
                    motif = seq2[im:im+kmer_length]
                    Counts[motif] += 1
    return Counts.most_common()


filed = "hg38.fasta"
sequenceO = mycustom.FastaFile(filed)
sequencesL = [ i.sequence.upper() for i in sequenceO ]
sequencesL_rev_compl = []
for i in sequencesL:
        seq = Seq(i)
        sequencesL_rev_compl+=[str(seq.reverse_complement())]

# kmer lengths to scan - here e scan 1bp to 15bp long kmers
kmer_lengthL=range(1,16)
Counts=fast_counter(sequencesL,sequencesL_rev_compl,kmer_lengthL)
	
datafile=open("hg38_data","w")
for i in Counts:
	datafile.write(str(i[0])+'\t'+str(i[1])+'\n')
datafile.close()	

In [ ]:
# nullomer_extraction/generate_all_counts.py

import re,os,sys,glob
import mycustom 
from Bio.Seq import Seq 
from collections import Counter
import pickle
import io


def reader(path):
        datafile=open(path,"r")
        data=datafile.readlines()
        datafile.close()
        Data=[]
        for i in data:
                Data+=[i.strip().split('\t')]
        return Data

DataL=reader('hg38_data')
DataD = {}
for i in DataL:
        if "N" not in i[0]:
                DataD[i[0]]=int(i[1])


DataL=reader("kmers_1_15nt")
Data2D={}
for k in DataL:
        Data2D[k[0]]=0

DataTD= { k: DataD.get(k, 0) + Data2D.get(k, 0) for k in set(DataD) | set(Data2D) }

datafile=open("hg38_data_total","w")
for i in DataTD.items():
        datafile.write(i[0]+'\t'+str(i[1])+'\n')
datafile.close()


In [ ]:
# nullomer_extraction/mycustom.py

class GenericFile(object):
    def __init__(self, fileNameS):
            try:
                fileF = open(fileNameS)
                self.fileS = fileF.read()
                fileF.close()
            except:
                self.fileS = fileNameS.read()

import pdb

class FastaFile(GenericFile):
    """
    This class represents a FastaFile.
    """

    def __init__(self, fasta, fileName=True):
        """

        :rtype : object
        :param fasta: This can be either a file name (string) or a HTML file upload element.
        :param fileName: This is a boolean value. If False, then it is the fasta string
        :return: does not return anything.
        """
        if fileName == True:
            super(FastaFile, self).__init__(fasta)
        elif fileName == False:
            if ">" not in fasta:
                raise ValueError("It is not in Fasta Format")
            self.fileS = fasta
        else:
            raise Exception

        self.sequences = None
        self.sequences = self.getSequences()


    def __getitem__(self, item):
        if self.sequences is None:
            self.getSequences()
        return self.sequences[item]

    def __len__(self):
        return len(self.getSequences())

    def __repr__(self):
        return "< Fasta file: %s sequences >" % len(self)

    def __str__(self):
        return "< Fasta file: %s sequences >" % len(self)

    def getSequences(self):
        if self.sequences is None:
            firstSplitL = self.fileS.strip().split(">")[1:]
            eachByLineL = [i.splitlines() for i in firstSplitL]

            sequences = []
            for i in eachByLineL:
                header = i[0]
                sequence = "".join(i[1:]).upper()
                sequence = sequence.replace(" ", "")
                sequence = sequence.replace("\t", "")
                if sequence == "":
                    raise ValueError("There is no sequence !")
                sequences.append(Sequence(name=header, sequence=sequence))

            self.sequences = sequences

        return self.sequences

    def getMaxLength(self):
        """
        This returns the max length of a sequence in this object
        """
        sequences = self.getSequences()
        maxLength = 0
        for sequence in sequences:
            seqLength = len(sequence)
            if seqLength > maxLength:
                maxLength = seqLength
        return maxLength

    def areDuplicatesPresent(self):
        sequences = self.getSequences()
        sequenceL = [i.sequence for i in sequences]
        if len(sequenceL) > len(set(sequenceL)):
            return True
        return False

    def getNames(self):

        names = [sequence.name for sequence in self.sequences]
        return names


class Sequence(object):
    """
    This class represents a DNA/RNA sequence.
    Each object needs to have a name.
    """
    def __init__(self, name, sequence):
        self.name = name
        self.sequence = sequence

    def __repr__(self):
        return "%s : %s" %(self.name, self.sequence)

    def __getitem__(self, item):
        return self.sequence[item]

    def __len__(self):
        return len(self.sequence)

    def __str__(self):
        return "%s : %s " % (self.name, self.sequence)



In [ ]:
# nullomer_extraction/possible_kmers.py import re,os,sys,itertools

def kmer_generate(lengthsL):
	total_kmers=[]
	bases=['A','T','G','C']
	for k in lengthsL:
		total_kmers+=[''.join(p) for p in itertools.product(bases, repeat=k)]
	return total_kmers

KmersL=kmer_generate(range(1,16))
		
datafile=open("kmers_1_15nt","w")
for i in KmersL:
        datafile.write(str(i)+'\n')
datafile.close()  
